In [37]:
import pandas as pd

df = pd.read_csv("Results - Sheet1.csv")

In [38]:
df.columns

Index(['Model', 'comments', 'dataset', 'dataset type', 'Precision Performance',
       'Recall Performance', 'F1 performance'],
      dtype='object')

In [39]:
df[["Model", "comments", "dataset", "dataset type"]] = df[
    ["Model", "comments", "dataset", "dataset type"]
].fillna("")
df[["Precision Performance", "Recall Performance", "F1 performance"]] = df[
    ["Precision Performance", "Recall Performance", "F1 performance"]
].fillna(0)

In [40]:
model = "bert-small"
comments = "attack on protocnn"

for attack in ["textfooler", "textbugger"]:
    print("Attack: ", attack)
    for dataset in ["imdb", "ag_news", "dbpedia"]:
        sub_df = df[
            (df["Model"] == model)
            & (df["comments"] == comments)
            & (df["dataset"] == dataset)
            & (df["dataset type"].str.endswith(attack))
        ]
        if len(sub_df) != 2:
            raise ValueError("Not 2 rows")
        adv = sub_df[sub_df["dataset type"] == f"adv - {attack}"]
        test = sub_df[sub_df["dataset type"] == f"test - {attack}"]
        print(
            "%.3f" % adv["Precision Performance"].values[0],
            "(",
            "%.3f"
            % (
                adv["Precision Performance"].values[0]
                - test["Precision Performance"].values[0]
            ),
            ")&",
            "%.3f" % adv["Recall Performance"].values[0],
            "(",
            "%.3f"
            % (
                adv["Recall Performance"].values[0]
                - test["Recall Performance"].values[0]
            ),
            ")&",
            "%.3f" % adv["F1 performance"].values[0],
            "(",
            "%.3f"
            % (adv["F1 performance"].values[0] - test["F1 performance"].values[0]),
            ")&",
        )
    print("------------------")

Attack:  textfooler
0.930 ( -0.031 )& 0.930 ( -0.031 )& 0.930 ( -0.031 )&
0.937 ( -0.013 )& 0.934 ( -0.014 )& 0.934 ( -0.014 )&
0.979 ( -0.015 )& 0.976 ( -0.018 )& 0.977 ( -0.017 )&
------------------
Attack:  textbugger
0.938 ( -0.021 )& 0.938 ( -0.021 )& 0.938 ( -0.021 )&
0.940 ( -0.011 )& 0.938 ( -0.011 )& 0.938 ( -0.011 )&
0.985 ( -0.009 )& 0.984 ( -0.010 )& 0.984 ( -0.010 )&
------------------
